In [1]:
import os
import torch
import pandas as pd

from torch.utils.data import DataLoader


from Hyperparameters.Embeddings.BertTokenEmbedder import BertTokenEmbedder
from Hyperparameters.Dataloader.EmbeddingDataset import EmbeddingDataset
from Hyperparameters.Dataloader.collate_fn import collate_fn

from Hyperparameters.Plotting.PlotEmbeddings import extract_all_embeddings_and_labels, datashader_plot, project_umap




C:\Users\Bruce\anaconda3\envs\CIL\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [6]:
seed = 42

## Model Parameters
prefix = "../"
model_name="FacebookAI/roberta-base"
csv_path= prefix + "data/Sentiment/training.csv"

In [3]:
embedder = BertTokenEmbedder(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

embedded_feature_dataset = None

if embedder.is_variable_length:

    cache_name= model_name.replace("/", "_")
    cache_path = prefix + "cache/" + cache_name
    emb_dataset_path = cache_path + "emb_dataset.pt"


    if os.path.exists(emb_dataset_path):
      
        embedded_feature_dataset = torch.load(emb_dataset_path, weights_only=False)
    else:
        
        df = pd.read_csv(csv_path, index_col=0)
        label_map = {'negative': -1, 'neutral': 0, 'positive': 1}
        df['label_encoded'] = df['label'].map(label_map)
        
        features = embedder.fit_transform(df['sentence'].to_list())
        labels = df['label_encoded'].to_numpy()

        feature_dataset = EmbeddingDataset(features, labels)
        
        feature_dataloader = DataLoader(feature_dataset, batch_size=8,collate_fn=collate_fn)
        embedded_feature_dataset = embedder.embed_dataset(feature_dataloader)
        os.makedirs("cache", exist_ok=True)
        torch.save(embedded_feature_dataset, emb_dataset_path)

else:
    raise Exception("blaalalal")

In [9]:
embs, labels = extract_all_embeddings_and_labels(embedded_feature_dataset)
embs_2d = project_umap(embs)
datashader_plot(embs_2d, labels)

Error accessing index 102097 - max index is 102096


C:\Users\Bruce\anaconda3\envs\CIL\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Bruce\anaconda3\envs\CIL\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


KeyboardInterrupt: 